In [4]:
from bs4 import BeautifulSoup
import urllib.request

urllib.request.urlretrieve("http://withdobong.org/files/attach/images/169/970/79c129fb7ffeb0929a40cdd510f5b4b3.jpg","./이미지 데이터셋/example1.png")

('./이미지 데이터셋/example1.png', <http.client.HTTPMessage at 0x19ecf37f748>)

In [2]:
import json

with open("./이미지 데이터셋/도봉서원_src.json",'r',encoding='utf-8') as makefile:
    data = json.load(makefile)

In [3]:
data

{'0': 'http://withdobong.org/files/attach/images/169/970/79c129fb7ffeb0929a40cdd510f5b4b3.jpg',
 '1': 'http://www.dbinews.com/data/2017/21031000/20170713_085914_6f80f30.png',
 '2': 'http://imguser2.pandora.tv/pandora/_channel_img_mp/y/u/yunhap/44/vod_thumb_51058644.jpg',
 '3': 'http://hanyangcho.net/admin/board2/upload_file/3p-2011%EB%8F%84%EB%B4%89%EC%84%9C%EC%9B%90_%EC%B6%98%ED%96%A5%EC%A0%9C.jpg',
 '4': 'http://www.report119.net/View/[%EC%82%AC%ED%9A%8C%EA%B3%BC%ED%95%99]%20%EB%8F%84%EB%B4%89%EC%84%9C%EC%9B%90%EC%A2%85%ED%95%A9%EB%B3%B5%EC%A7%80%EA%B4%80%EA%B8%B0%EA%B4%80%EB%B6%84%EC%84%9D%EB%B3%B4%EA%B3%A0%EC%84%9C_hwp_01.gif',
 '5': 'http://ceo.dobong.go.kr/FILE/bbs/bcode346/3D43E81B-FDCE-1E21-D1C6-30F18713F0FC.JPG',
 '6': 'http://www.kimseondong.com/uploads/cache/editor/2016/01/thumb-f7cc6b6f4a90128110688e8dc5f570d6_960x0.jpg',
 '7': 'https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=518866004891184',
 '8': 'http://img.hani.co.kr/imgdb/resize/2009/0618/6000257457_2009

In [5]:
import urllib.request
for i in range(0,579):
    try:
        urllib.request.urlretrieve(data[str(i)], './이미지 데이터셋/도봉서원/'+str(i)+'.jpg')
    except:
        print(i)

34
117
163
183
241
252
253
272
282
337
378
411


In [7]:
with open("./이미지 데이터셋/봉황각_src.json",'r',encoding='utf-8') as makefile:
    data2 = json.load(makefile)

for i in range(0,610):
    try:
        urllib.request.urlretrieve(data2[str(i)], './이미지 데이터셋/봉황각/'+str(i)+'.jpg')
    except:
        print(i)    
    

12
15
23
47
69
81
91
114
128
149
165
196
207
208
224
235
240
267
270
292
299
365
370
407
414
416
417
419
430
443
463
502
517
522
586


In [9]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

Using TensorFlow backend.


In [13]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

img = load_img('./이미지 데이터셋/봉황각/6.jpg')  # PIL 이미지
x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열

# 아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
# 지정된 `preview/` 폴더에 저장합니다.
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='./이미지 데이터셋/봉황각/봉황각2', save_prefix='example', save_format='jpeg'):
    i += 1
    if i > 30:
        break

In [52]:
#모델링
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Conv2D(128, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
batch_size = 16

# 학습 이미지에 적용한 augmentation 인자를 지정해줍니다.
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# 검증 및 테스트 이미지는 augmentation을 적용하지 않습니다. 모델 성능을 평가할 때에는 이미지 원본을 사용합니다.
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow_from_directory(
        './img/train',  # this is the target directory
        target_size=(150, 150),  # 모든 이미지의 크기가 150x150로 조정됩니다.
        batch_size=batch_size,
        class_mode='binary')  # binary_crossentropy 손실 함수를 사용하므로 binary 형태로 라벨을 불러와야 합니다.

validation_generator = validation_datagen.flow_from_directory(
        './img/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        './img/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 138 images belonging to 2 classes.
Found 53 images belonging to 2 classes.
Found 53 images belonging to 2 classes.


In [60]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.metrics import categorical_accuracy, categorical_crossentropy
from tensorflow.keras.optimizers import Adam

model = MobileNetV2(input_shape=(150, 150, 3), alpha=1., weights=None, classes=2)
model.compile(optimizer=Adam(lr=0.002), loss='sparse_categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy])

In [55]:
train_generator

In [61]:
model.fit_generator(
        train_generator,
        steps_per_epoch=1000 // batch_size,
        validation_data=validation_generator,
        validation_steps=10,
        epochs=50)
model.save_weights('first_try.h5') 

Epoch 1/50
62/62 [==============================] - 152s 2s/step - loss: 0.7546 - categorical_crossentropy: 0.6066 - categorical_accuracy: 0.8800 - val_loss: 0.6779 - val_categorical_crossentropy: 0.4330 - val_categorical_accuracy: 1.0000
Epoch 2/50
62/62 [==============================] - 145s 2s/step - loss: 0.5269 - categorical_crossentropy: 0.4354 - categorical_accuracy: 0.8705 - val_loss: 0.6185 - val_categorical_crossentropy: 0.4367 - val_categorical_accuracy: 1.0000
Epoch 3/50
62/62 [==============================] - 149s 2s/step - loss: 0.4738 - categorical_crossentropy: 0.4123 - categorical_accuracy: 0.9189 - val_loss: 0.6321 - val_categorical_crossentropy: 0.4587 - val_categorical_accuracy: 1.0000
Epoch 4/50
62/62 [==============================] - 154s 2s/step - loss: 0.4357 - categorical_crossentropy: 0.3963 - categorical_accuracy: 0.7758 - val_loss: 0.6258 - val_categorical_crossentropy: 0.4355 - val_categorical_accuracy: 1.0000
Epoch 5/50
62/62 [==========================

Epoch 35/50
62/62 [==============================] - 132s 2s/step - loss: 0.3539 - categorical_crossentropy: 0.3750 - categorical_accuracy: 0.5811 - val_loss: 0.7142 - val_categorical_crossentropy: 0.4227 - val_categorical_accuracy: 0.0000e+00
Epoch 36/50
62/62 [==============================] - 132s 2s/step - loss: 0.3393 - categorical_crossentropy: 0.3644 - categorical_accuracy: 0.5379 - val_loss: 0.7261 - val_categorical_crossentropy: 0.4137 - val_categorical_accuracy: 0.0000e+00
Epoch 37/50
62/62 [==============================] - 132s 2s/step - loss: 0.3361 - categorical_crossentropy: 0.3677 - categorical_accuracy: 0.5379 - val_loss: 0.7237 - val_categorical_crossentropy: 0.4135 - val_categorical_accuracy: 0.0000e+00
Epoch 38/50
62/62 [==============================] - 134s 2s/step - loss: 0.3404 - categorical_crossentropy: 0.3660 - categorical_accuracy: 0.6527 - val_loss: 0.7288 - val_categorical_crossentropy: 0.4240 - val_categorical_accuracy: 0.0000e+00
Epoch 39/50
62/62 [=====

In [44]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_24 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_25 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 34, 34, 64)        18496     
__________